# Real Time Tribe

The `RealTimeTribe` class provides (near) real-time matched-filter methods.  It subclasses EQcorrscan's `Tribe` class and therefore
provides all the same methods, as well as real-time methods. We simply make a `RealTimeTribe` from a `Tribe` and some streaming client.
RT-EQcorrscan provides a `_Streaming` abstract base class outlining the basic attributes and methods a streaming client needs to have for
real-time matched-filtering.  The `RealTimeClient` is a mix of obspy's `EasySeedLinkClient` and the `_Streaming` abc, providing
access to seedlink services for real-time matched-filtering.

In this example we will see what seismicity is going on associated with the Rigdecrest sequence in California.

In [1]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from eqcorrscan import Tribe
from eqcorrscan.utils.catalog_utils import filter_picks
from rt_eqcorrscan import RealTimeTribe
from rt_eqcorrscan.streaming import RealTimeClient

client = Client("NCEDC")  # IRIS doesn't provide picks
rt_client = RealTimeClient(
    server_url="rtserve.iris.washington.edu",
    buffer_capacity=600.) # NCEDC doesn't provide a public seedlink

template_catalog = client.get_events(
    latitude=35.7, longitude=-117.4, maxradius=0.2, 
    starttime=UTCDateTime(2019, 7, 1), endtime=UTCDateTime(2019, 7, 15), 
    minmagnitude=4.5, includearrivals=True)
_ = template_catalog.plot(projection="local")

<Figure size 640x480 with 2 Axes>

We first have to generate a `Tribe` from this catalog. This will take a little while, and is one of the key motivations for using a `TemplateBank` of
previously recorded earthquakes.

For speed we will filter the catalog

In [2]:
filtered_catalog = filter_picks(
    template_catalog, top_n_picks=5, 
    networks=["PB", "LB", "US"])
tribe = Tribe().construct(
    method="from_client", catalog=filtered_catalog, client_id=Client("IRIS"),
    lowcut=2., highcut=15., samp_rate=50., filt_order=4, prepick=0.5, length=4.,
    swin="all", process_len=300.)

/home/chambeca/miniconda3/envs/eqcorrscan/lib/python3.7/site-packages/obspy/signal/detrend.py:31: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(data.dtype, float):
Pick for TPH.HHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for B076.EHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for B079.EHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for B900.EHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for B901.EHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for TPH.HHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for TPH.HHZ has n

We also need an inventory of stations that will be used.

In [3]:
seed_ids = {tuple(pick.waveform_id.get_seed_string().split('.'))
            for event in filtered_catalog for pick in event.picks}
bulk = [
    (sid[0], sid[1], sid[2], sid[3], UTCDateTime(2019, 7, 1), 
     UTCDateTime(2019, 7, 10)) for sid in seed_ids]
inventory = client.get_stations_bulk(bulk, level="channel")

Finally we can make the real-time tribe (note that plotting isn't supported in the notebook, but if you run this outside of the notebook and turn `plot=True` then you will get a bokeh live updating plot:

In [4]:
rt_tribe = RealTimeTribe(
        tribe=tribe, inventory=inventory, rt_client=rt_client,
        detect_interval=20, plot=False, plot_options=dict(plot_length=600.))

And we can run the (near) real-time detection!

In [5]:
import logging

logging.basicConfig(
    level="INFO", format="%(asctime)s\t%(name)s\t%(levelname)s\t%(message)s"
)

rt_tribe.run(
    threshold=0.25, threshold_type="av_chan_corr", trig_int=10., 
    max_run_length=1200)  # Max_run_length will stop the process - leave unset to run forever.

2019-07-18 17:55:35,560	 obspy.clients.seedlink [rtserve.iris.washington.edu:18000]	INFO	sending: requesting INFO level CAPABILITIES
2019-07-18 17:55:35,743	rt_eqcorrscan.streaming.streaming	INFO	Started streaming
2019-07-18 17:55:35,743	rt_eqcorrscan.rt_match_filter	INFO	Started real-time streaming
2019-07-18 17:55:35,744	rt_eqcorrscan.rt_match_filter	INFO	Detection will use the following data: {'LB.TPH.01.HHZ', 'PB.B900..EHZ', 'PB.B901..EHZ', 'PB.B079..EHZ', 'PB.B076..EHZ'}
2019-07-18 17:55:35,745	rt_eqcorrscan.rt_match_filter	INFO	Sleeping for 605.00s while accumulating data
2019-07-18 17:55:36,102	 obspy.clients.seedlink [rtserve.iris.washington.edu:18000]	INFO	requesting next available data
2019-07-18 17:55:36,641	 obspy.clients.seedlink [rtserve.iris.washington.edu:18000]	INFO	requesting next available data
2019-07-18 17:55:37,182	 obspy.clients.seedlink [rtserve.iris.washington.edu:18000]	INFO	requesting next available data
2019-07-18 17:55:37,722	 obspy.clients.seedlink [rtserv

2019-07-18 18:06:01,167	eqcorrscan.core.match_filter.matched_filter	WARNING	Last bit of data between 2019-07-18T06:05:18.738400Z and 2019-07-18T06:05:19.748300Z will go unused because it is shorter than a chunk of 300.0 s
2019-07-18 18:06:01,168	eqcorrscan.core.match_filter.matched_filter	INFO	Computing detections between 2019-07-18T05:55:28.718300Z and 2019-07-18T06:00:28.698300Z
2019-07-18 18:06:01,194	eqcorrscan.core.match_filter.matched_filter	INFO	Starting the correlation run for these data
2019-07-18 18:06:01,264	eqcorrscan.core.match_filter.matched_filter	INFO	Looping over templates and streams took: 0.0674s
2019-07-18 18:06:01,269	eqcorrscan.core.match_filter.matched_filter	INFO	Finding peaks took 0.0039s
2019-07-18 18:06:01,292	eqcorrscan.core.match_filter.matched_filter	INFO	Made 48 detections from 11 templates
2019-07-18 18:06:01,293	eqcorrscan.core.match_filter.matched_filter	INFO	Computing detections between 2019-07-18T06:00:18.738300Z and 2019-07-18T06:05:18.718300Z
2019-

/home/chambeca/miniconda3/envs/eqcorrscan/lib/python3.7/site-packages/obspy/signal/detrend.py:31: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(data.dtype, float):
/home/chambeca/miniconda3/envs/eqcorrscan/lib/python3.7/site-packages/obspy/signal/detrend.py:31: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(data.dtype, float):
/home/chambeca/miniconda3/envs/eqcorrscan/lib/python3.7/site-packages/obspy/signal/detrend.py:31: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(data.dtype, float):
/home/chambeca/miniconda3/envs/eqcorrsca

2019-07-18 18:07:41,100	eqcorrscan.core.match_filter.matched_filter	WARNING	Last bit of data between 2019-07-18T06:02:18.268300Z and 2019-07-18T06:06:44.658300Z will go unused because it is shorter than a chunk of 300.0 s
2019-07-18 18:07:41,100	eqcorrscan.core.match_filter.matched_filter	INFO	Computing detections between 2019-07-18T05:57:18.268300Z and 2019-07-18T06:02:18.248300Z
2019-07-18 18:07:41,124	eqcorrscan.core.match_filter.matched_filter	INFO	Starting the correlation run for these data
2019-07-18 18:07:41,196	eqcorrscan.core.match_filter.matched_filter	INFO	Looping over templates and streams took: 0.0688s
2019-07-18 18:07:41,199	eqcorrscan.core.match_filter.matched_filter	INFO	Finding peaks took 0.0026s
2019-07-18 18:07:41,221	eqcorrscan.core.match_filter.matched_filter	INFO	Made 50 detections from 11 templates
2019-07-18 18:07:41,265	rt_eqcorrscan.rt_match_filter	INFO	Removing duplicate detections
2019-07-18 18:07:41,271	rt_eqcorrscan.rt_match_filter	INFO	Party now contains 

2019-07-18 18:08:01,673	rt_eqcorrscan.rt_match_filter	INFO	Detection took 0.71s
2019-07-18 18:08:01,674	rt_eqcorrscan.rt_match_filter	INFO	Waiting 19.29s until next run
2019-07-18 18:08:20,987	rt_eqcorrscan.rt_match_filter	INFO	Starting detection run
/home/chambeca/miniconda3/envs/eqcorrscan/lib/python3.7/site-packages/obspy/signal/detrend.py:31: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(data.dtype, float):
/home/chambeca/miniconda3/envs/eqcorrscan/lib/python3.7/site-packages/obspy/signal/detrend.py:31: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(data.dtype, float):
/home/chambeca/miniconda3/envs/eqcorrscan/lib/python3.7/site-packages/obspy/signal/detrend.py:31: FutureWarni

...

2019-07-18 18:12:41,375	eqcorrscan.core.match_filter.matched_filter	WARNING	Last bit of data between 2019-07-18T06:07:19.948300Z and 2019-07-18T06:11:57.688300Z will go unused because it is shorter than a chunk of 300.0 s
2019-07-18 18:12:41,376	eqcorrscan.core.match_filter.matched_filter	INFO	Computing detections between 2019-07-18T06:02:19.948300Z and 2019-07-18T06:07:19.928300Z
2019-07-18 18:12:41,401	eqcorrscan.core.match_filter.matched_filter	INFO	Starting the correlation run for these data
2019-07-18 18:12:41,481	eqcorrscan.core.match_filter.matched_filter	INFO	Looping over templates and streams took: 0.0765s
2019-07-18 18:12:41,485	eqcorrscan.core.match_filter.matched_filter	INFO	Finding peaks took 0.0034s
2019-07-18 18:12:41,514	eqcorrscan.core.match_filter.matched_filter	INFO	Made 62 detections from 11 templates
2019-07-18 18:12:41,567	rt_eqcorrscan.rt_match_filter	INFO	Removing duplicate detections
2019-07-18 18:12:41,584	rt_eqcorrscan.rt_match_filter	INFO	Party now contains 

2019-07-18 18:13:21,457	eqcorrscan.utils.pre_processing	INFO	Data for LB.TPH.02.HHZ are not long-enough, will zero pad
2019-07-18 18:13:21,457	eqcorrscan.utils.pre_processing	INFO	Data for LB.TPH.01.HHZ are not long-enough, will zero pad
2019-07-18 18:13:21,459	eqcorrscan.utils.pre_processing	INFO	Data for PB.B901..EHZ are not long-enough, will zero pad
2019-07-18 18:13:21,459	eqcorrscan.utils.pre_processing	INFO	Data for PB.B900..EHZ are not long-enough, will zero pad
2019-07-18 18:13:21,458	eqcorrscan.utils.pre_processing	INFO	Data for PB.B076..EHZ are not long-enough, will zero pad
2019-07-18 18:13:21,459	eqcorrscan.utils.pre_processing	INFO	Data for PB.B079..EHZ are not long-enough, will zero pad
2019-07-18 18:13:21,559	eqcorrscan.core.match_filter.matched_filter	INFO	Computing detections between 2019-07-18T06:02:55.878300Z and 2019-07-18T06:07:55.858300Z
2019-07-18 18:13:21,586	eqcorrscan.core.match_filter.matched_filter	INFO	Starting the correlation run for these data
2019-07-18 

2019-07-18 18:14:01,463	eqcorrscan.core.match_filter.matched_filter	WARNING	Last bit of data between 2019-07-18T06:08:20.608300Z and 2019-07-18T06:12:54.588300Z will go unused because it is shorter than a chunk of 300.0 s
2019-07-18 18:14:01,464	eqcorrscan.core.match_filter.matched_filter	INFO	Computing detections between 2019-07-18T06:03:20.608300Z and 2019-07-18T06:08:20.588300Z
2019-07-18 18:14:01,490	eqcorrscan.core.match_filter.matched_filter	INFO	Starting the correlation run for these data
2019-07-18 18:14:01,563	eqcorrscan.core.match_filter.matched_filter	INFO	Looping over templates and streams took: 0.0696s
2019-07-18 18:14:01,567	eqcorrscan.core.match_filter.matched_filter	INFO	Finding peaks took 0.0030s
2019-07-18 18:14:01,599	eqcorrscan.core.match_filter.matched_filter	INFO	Made 70 detections from 11 templates
2019-07-18 18:14:01,658	rt_eqcorrscan.rt_match_filter	INFO	Removing duplicate detections
2019-07-18 18:14:01,672	rt_eqcorrscan.rt_match_filter	INFO	Party now contains 

/home/chambeca/miniconda3/envs/eqcorrscan/lib/python3.7/site-packages/obspy/signal/detrend.py:31: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(data.dtype, float):
/home/chambeca/miniconda3/envs/eqcorrscan/lib/python3.7/site-packages/obspy/signal/detrend.py:31: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(data.dtype, float):
/home/chambeca/miniconda3/envs/eqcorrscan/lib/python3.7/site-packages/obspy/signal/detrend.py:31: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(data.dtype, float):
/home/chambeca/miniconda3/envs/eqcorrsca

Party of 4 Families.

This will store detections in a `detections` directory.